In [2]:
import numpy as np
import nnetwork
import mnist_loader

In [3]:
training_data, validation_data, test_data = mnist_loader.load_data_wrapper()

In [5]:
net = nnetwork.Network([784, 30, 10])

In [6]:
net.SGD(training_data, 30, 10, 3.0, test_data=test_data)

Epoch 0: 8952 / 10000
Epoch 1: 9201 / 10000
Epoch 2: 9284 / 10000
Epoch 3: 9375 / 10000
Epoch 4: 9377 / 10000
Epoch 5: 9410 / 10000
Epoch 6: 9433 / 10000
Epoch 7: 9432 / 10000
Epoch 8: 9452 / 10000
Epoch 9: 9451 / 10000
Epoch 10: 9452 / 10000
Epoch 11: 9489 / 10000
Epoch 12: 9456 / 10000
Epoch 13: 9488 / 10000
Epoch 14: 9474 / 10000
Epoch 15: 9455 / 10000
Epoch 16: 9449 / 10000
Epoch 17: 9448 / 10000
Epoch 18: 9464 / 10000
Epoch 19: 9462 / 10000
Epoch 20: 9487 / 10000
Epoch 21: 9482 / 10000
Epoch 22: 9486 / 10000
Epoch 23: 9464 / 10000
Epoch 24: 9476 / 10000
Epoch 25: 9471 / 10000
Epoch 26: 9469 / 10000
Epoch 27: 9457 / 10000
Epoch 28: 9463 / 10000
Epoch 29: 9499 / 10000


In [20]:
print'Entrenamiento usando :'
print 'learning rate = %s' % 3.0
print 'epochs = %s' % 30
print 'batch size = %s' % 10
print'Presicion = ' + str(float(net.evaluate(test_data))/10000*100) + '%'

Entrenamiento usando :
learning rate = 3.0
epochs = 30
batch size = 10
Presicion = 94.99%
